# PGFinder Interactive Notebook

This notebook is a basic user interface to allow researchers less familiar with the command line to run PGFinder. Some compromises have been made to make it run as a Jupyter notebook on the free myBinder platform.

To use the code to analyse your data you must work from top to bottom on this notebook, following instructions as you go. The first step is to execute all of the "cells" in the notebook so they are ready for your input. To do this, click *Kernel*>*Restart & Run All* on the menu, above.

# Setup

In [ ]:
import base64
import codecs
import io
import os
import ipywidgets as widgets
from ipywidgets import HTML
from IPython.display import display
from ipysheet import from_dataframe
import pandas as pd

from pgfinder import matching, validation

# Get list of modifications
allowed_mods = validation.allowed_modifications()

# Get built in mass lists
mass_lists_path = './data/masses'
mass_lists = os.listdir(mass_lists_path)

# Utility functions
def safe_load_maxquant():
    try:
        file_contents = uploader.value[list(uploader.value.keys())[0]]['content'] # I hate this line of code
        return matching.maxquant_file_reader(io.BytesIO(file_contents))
        # display(from_dataframe(uploaded_df.head(10)))
    except:
        print("Could not upload and display file, please make sure you have selected a file.")

# Main analysis function
def analysis(b):
    # Upload maxquant file
    uploaded_df = safe_load_maxquant()
    
    # Load mass list
    csv_filepath = os.path.join(mass_lists_path, rb_masses.value)
    theo_masses = matching.theo_masses_reader(csv_filepath)
    
    # Make sure mod list is a list
    mod_list = list(selector_mods.value)
    
    # Validate inputs
    validation.validate_raw_data_df(uploaded_df)
    validation.validate_theo_masses_df(theo_masses)
    validation.validate_enabled_mod_list(mod_list)

    results = matching.data_analysis(uploaded_df, theo_masses, 0.5, mod_list, 10)
    
    # Make the download button
    results_csv_str = results.to_csv()
    filename = 'results.csv'
    b64 = base64.b64encode(results_csv_str.encode())
    payload = b64.decode()

    html_buttons = '''<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/csv;base64,{payload}" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
    </a>
    </body>
    </html>
    '''

    html_button = html_buttons.format(payload=payload,filename=filename)
    display(HTML(html_button))
    
# Define widgets

# Maxquant file upload
uploader = widgets.FileUpload(
    accept = '.txt', 
    description = 'Upload Maxquant file',
    multiple = False
) 

# Modifcation selector
selector_mods = widgets.SelectMultiple(
    options = allowed_mods,
    description = 'Modification',
    disabled = False
)

# Mass list selector
rb_masses = widgets.RadioButtons(
    options = mass_lists,
    description = 'Mass List',
    disabled = False
)

# Analysis button
button = widgets.Button(description="Run Analysis")
button.on_click(analysis)

## User Interface

1. Click *Upload* to upload a `.txt` file output by MaxQuant.
2. Select modifications (Hold down control / command and click to select mulitple items.)
3. Select mass list.
4. Click run analysis.
5. AFter the analysis is complete, a download button will appear.

In [ ]:
display(uploader)
display(selector_mods) 
display(rb_masses)
display(button)